In [2]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import scatter_matrix

import sklearn
from sklearn.model_selection import KFold, cross_validate, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split # Split data
from sklearn.metrics import accuracy_score # Compute accuracy
from sklearn.metrics import f1_score

from sklearn.decomposition import PCA 

# Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC


import random
from tqdm import tqdm # Progress Bar

from sklearn.preprocessing import LabelEncoder



# Options
pd.set_option("display.precision", 2)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
def generate_synthetic_data(df, columns, num_new_instances):
  '''
  Creates new synthetic observations with noise added 

  @arguments df - dataframe
             columns - column indexes we want to change
             num_new_instances - number of new observations to create

  @return df - new dataframe with new observations

    
  @author Conrad Pereira

  '''
  noise = np.random.normal(0, 1, num_new_instances)
  indices = [random.randint(0, 91212) for i in range(num_new_instances)]

  print("========= Creating New Data =========")
  for idx in tqdm(indices):
    df = df.append(df.iloc[idx], ignore_index=True)
  
  print("========= Adding Noise =========")
  for row in tqdm(range(len(df) - num_new_instances, len(df))):
    for col in columns:
      df.iat[row, col] += random.choice(noise)
  return df

In [4]:
preprocessed_df = pd.read_csv("../Data/preprocessed_data.csv")

preprocessed_df

,track_id,title,title.1,location,name,bit_rate,genre_id,kurtosis,kurtosis.1,kurtosis.2,kurtosis.3,kurtosis.4,kurtosis.5,kurtosis.6,kurtosis.7,kurtosis.8,kurtosis.9,kurtosis.10,kurtosis.11,max,max.1,max.2,max.3,max.4,max.5,max.6,max.7,max.8,max.9,max.10,max.11,mean,mean.1,mean.2,mean.3,mean.4,mean.5,mean.6,mean.7,mean.8,mean.9,mean.10,mean.11,median,median.1,median.2,median.3,median.4,median.5,median.6,median.7,median.8,median.9,median.10,median.11,min,min.1,min.2,min.3,min.4,min.5,min.6,min.7,min.8,min.9,min.10,min.11,skew,skew.1,skew.2,skew.3,skew.4,skew.5,skew.6,skew.7,skew.8,skew.9,skew.10,skew.11,std,std.1,std.2,std.3,std.4,std.5,std.6,std.7,std.8,std.9,std.10,std.11,kurtosis.12,kurtosis.13,kurtosis.14,kurtosis.15,kurtosis.16,kurtosis.17,kurtosis.18,kurtosis.19,kurtosis.20,kurtosis.21,kurtosis.22,kurtosis.23,max.12,max.13,max.14,max.15,max.16,max.17,max.18,max.19,max.20,max.21,max.22,max.23,mean.12,mean.13,mean.14,mean.15,mean.16,mean.17,mean.18,mean.19,mean.20,mean.21,mean.22,mean.23,median.12,median.13,median.14,median.15,median.16,median.17,median.18,median.19,median.20,median.21,median.22,median.23,min.12,min.13,min.14,min.15,min.16,min.17,min.18,min.19,min.20,min.21,min.22,min.23,skew.12,skew.13,skew.14,skew.15,skew.16,skew.17,skew.18,skew.19,skew.20,skew.21,skew.22,skew.23,std.12,std.13,std.14,std.15,std.16,std.17,std.18,std.19,std.20,std.21,std.22,std.23,kurtosis.24,kurtosis.25,kurtosis.26,kurtosis.27,kurtosis.28,kurtosis.29,kurtosis.30,kurtosis.31,kurtosis.32,kurtosis.33,kurtosis.34,kurtosis.35,max.24,max.25,max.26,max.27,max.28,max.29,max.30,max.31,max.32,max.33,max.34,max.35,mean.24,mean.25,mean.26,mean.27,mean.28,mean.29,mean.30,mean.31,mean.32,mean.33,mean.34,mean.35,median.24,median.25,median.26,median.27,median.28,median.29,median.30,median.31,median.32,median.33,median.34,median.35,min.24,min.25,min.26,min.27,min.28,min.29,min.30,min.31,min.32,min.33,min.34,min.35,skew.24,skew.25,skew.26,skew.27,skew.28,skew.29,skew.30,skew.31,skew.32,skew.33,skew.34,skew.35,std.24,std.25,std.26,std.27,std.28,std.29,std.30,std.31,std.32,std.33,std.34,std.35,kurtosis.36,kurtosis.37,kurtosis.38,kurtosis.39,kurtosis.40,kurtosis.41,kurtosis.42,kurtosis.43,kurtosis.44,kurtosis.45,kurtosis.46,kurtosis.47,kurtosis.48,kurtosis.49,kurtosis.50,kurtosis.51,kurtosis.52,kurtosis.53,kurtosis.54,kurtosis.55,max.36,max.37,max.38,max.39,max.40,max.41,max.42,max.43,max.44,max.45,max.46,max.47,max.48,max.49,max.50,max.51,max.52,max.53,max.54,max.55,mean.36,mean.37,mean.38,mean.39,mean.40,mean.41,mean.42,mean.43,mean.44,mean.45,mean.46,mean.47,mean.48,mean.49,mean.50,mean.51,mean.52,mean.53,mean.54,mean.55,median.36,median.37,median.38,median.39,median.40,median.41,median.42,median.43,median.44,median.45,median.46,median.47,median.48,median.49,median.50,median.51,median.52,median.53,median.54,median.55,min.36,min.37,min.38,min.39,min.40,min.41,min.42,min.43,min.44,min.45,min.46,min.47,min.48,min.49,min.50,min.51,min.52,min.53,min.54,min.55,skew.36,skew.37,skew.38,skew.39,skew.40,skew.41,skew.42,skew.43,skew.44,skew.45,skew.46,skew.47,skew.48,skew.49,skew.50,skew.51,skew.52,skew.53,skew.54,skew.55,std.36,std.37,std.38,std.39,std.40,std.41,std.42,std.43,std.44,std.45,std.46,std.47,std.48,std.49,std.50,std.51,std.52,std.53,std.54,std.55,kurtosis.56,max.56,mean.56,median.56,min.56,skew.56,std.56,kurtosis.57,max.57,mean.57,median.57,min.57,skew.57,std.57,kurtosis.58,max.58,mean.58,median.58,min.58,skew.58,std.58,kurtosis.59,kurtosis.60,kurtosis.61,kurtosis.62,kurtosis.63,kurtosis.64,kurtosis.65,max.59,max.60,max.61,max.62,max.63,max.64,max.65,mean.59,mean.60,mean.61,mean.62,mean.63,mean.64,mean.65,median.59,median.60,median.61,median.62,median.63,median.64,median.65,min.59,min.60,min.61,min.62,min.63,min.64,min.65,skew.59,skew.60,skew.61,skew.62,skew.63,skew.64,skew.65,std.59,std.60,std.61,std.62,std.63,std.64,std.65,kurtosis.66,max.66,mean.66,median.66,min.66,skew.66,std.66,kurtosis.67,kurtosis.68,kurtosis.69,kurtosis.70,kurtosis.71,kurtosis.72,max

In [6]:
# Selecting the numeric columns that make sense for now... (not ID, etc.) 
column_idx = [i for i in range(7, len(preprocessed_df.columns) - 1)]

preprocessed_augmented_df = generate_synthetic_data(preprocessed_df, column_idx, 100000)

  0%|          | 0/10000 [00:00<?, ?it/s]

========= Creating New Data =========


  0%|          | 2/10000 [00:00<10:07, 16.45it/s]

========= Adding Noise =========


100%|██████████| 10000/10000 [09:00<00:00, 18.49it/s]


In [8]:
preprocessed_augmented_df.to_csv(r'..\Data\preprocessed_synthetic_data_fma.csv', index=False)